In [1]:
import calendar
import pandas as pd
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
    'qty':'{:,}','price':'{:.2f}','target':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}
cols = 'trade name qty price active reason market'.split()
colt = 'trans name qty target active spd'.split()
cash = 720_000
cash

720000

### Print to verify before upload file

In [53]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [54]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

orders[cols].to_csv(output_file, header=False, index=False)
orders[cols].to_csv(data_file, header=False, index=False)
orders[cols].to_csv(box_file, header=False, index=False)

### End of transactional process

In [55]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,BGC,"10,000",10.40,"104,000.00",DOS,SET,2
1,B,DOHOME,"4,000",22.90,"91,600.00",RD05%,SET100,2
2,B,IMH,"6,000",12.00,"72,000.00",RU09%,mai,2
3,B,KCE,800,85.50,"68,400.00",RD02%,SET50,2
4,B,WHART,"10,000",12.50,"125,000.00",DOS,SET,2
5,S,DCC,"40,000",3.00,"120,000.00",3%,SET,2


In [56]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    461000.0
S    120000.0
Name: amount, dtype: float64

In [57]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,BGC,"10,000",10.40,"104,000.00",DOS,SET,2
1,B,DOHOME,"4,000",22.90,"91,600.00",RD05%,SET100,2
2,B,IMH,"6,000",12.00,"72,000.00",RU09%,mai,2
3,B,KCE,800,85.50,"68,400.00",RD02%,SET50,2
4,B,WHART,"10,000",12.50,"125,000.00",DOS,SET,2


In [58]:
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(720000, 461000.0, 259000.0)

In [59]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
5,S,DCC,"40,000",3.00,"120,000.00",3%,SET,2


In [60]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(720000, 120000.0, 840000.0)

### After call ord-log

In [61]:
def categorise(row):  
    if row['col0'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [62]:
df = pd.read_csv('../data/orders-log.csv', header=None, prefix = 'col',usecols=[0,1,2,3,4,5,6,7,8,9,11])
df.rename(columns = {'col1': 'name', 'col2': 'spd','col3': 'reason', 'col4': 'market','col5': 'qty', 'col6': 'target','col7': 'current', 'col8': 'chg', 'col9': 'percent', 'col11': 'active'}, inplace = True)
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [63]:
show(df[['trans','name','spd','reason','qty','target','amount','current','active']])

trans,name,spd,reason,qty,target,amount,current,active


In [65]:
mb = (df.trans == 'Buy') & (df.spd >= -4)
df[mb].sort_values(['name'],ascending=[True])

,col0,name,spd,reason,market,qty,target,current,chg,percent,active,trans,amount
0,B,BGC,-3,DOS,SET,10000,10.4,10.70,0.10,+0.94%,2,Buy,104000.0
1,B,DOHOME,-4,RD05%,SET100,4000,22.9,23.30,0.00,0.00%,2,Buy,91600.0
8,B,ROJNA,-4,6L,SET,18000,6.5,6.70,0.05,+0.75%,1,Buy,117000.0
14,B,TVO,-3,DOS,SET,3600,30.0,30.75,-0.25,-0.81%,1,Buy,108000.0
15,B,WHART,-2,DOS,SET,10000,12.5,12.70,0.00,0.00%,2,Buy,125000.0


In [66]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [67]:
ms = (df.trans == 'Sell') & (df.spd <= 5)
df[ms].sort_values(['name'],ascending=[True])

,col0,name,spd,reason,market,qty,target,current,chg,percent,active,trans,amount
17,S,DCC,3,3%,SET,40000,3.0,2.94,0.10,+3.52%,2,Sell,120000.0
18,S,IVL,2,5%,SET50,3000,44.0,43.50,0.25,+0.58%,1,Sell,132000.0
21,S,SAT,5,10%,SET,2500,23.2,22.70,0.90,+4.13%,1,Sell,58000.0


In [68]:
df[mb | ms].sort_values(['col0','name'],ascending=[True,True])

,col0,name,spd,reason,market,qty,target,current,chg,percent,active,trans,amount
0,B,BGC,-3,DOS,SET,10000,10.4,10.70,0.10,+0.94%,2,Buy,104000.0
1,B,DOHOME,-4,RD05%,SET100,4000,22.9,23.30,0.00,0.00%,2,Buy,91600.0
8,B,ROJNA,-4,6L,SET,18000,6.5,6.70,0.05,+0.75%,1,Buy,117000.0
14,B,TVO,-3,DOS,SET,3600,30.0,30.75,-0.25,-0.81%,1,Buy,108000.0
15,B,WHART,-2,DOS,SET,10000,12.5,12.70,0.00,0.00%,2,Buy,125000.0
17,S,DCC,3,3%,SET,40000,3.0,2.94,0.10,+3.52%,2,Sell,120000.0
18,S,IVL,2,5%,SET50,3000,44.0,43.50,0.25,+0.58%,1,Sell,132000.0
21,S,SAT,5,10%,SET,2500,23.2,22.70,0.90,+4.13%,1,Sell,58000.0


In [69]:
df[mb | ms].shape[0]

8

In [70]:
mask = (df.col0 == 'B')
df[mb | ms][mask].nlargest(5, "amount")[["name","trans","amount"]].style.format(format_dict)

C:\Users\User\AppData\Local\Temp/ipykernel_4368/1898287905.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[mb | ms][mask].nlargest(5, "amount")[["name","trans","amount"]].style.format(format_dict)


,name,trans,amount
15,WHART,Buy,"125,000.00"
8,ROJNA,Buy,"117,000.00"
14,TVO,Buy,"108,000.00"
0,BGC,Buy,"104,000.00"
1,DOHOME,Buy,"91,600.00"


In [71]:
#df_out = df[mb | ms][colt]
df_out = df[df.active == 2][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd
0,Buy,BGC,"10,000",10.40,2,-3
1,Buy,DOHOME,"4,000",22.90,2,-4
3,Buy,IMH,"6,000",12.00,2,-5
4,Buy,KCE,800,85.50,2,-12
15,Buy,WHART,"10,000",12.50,2,-2
17,Sell,DCC,"40,000",3.00,2,3


In [72]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=False, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=False, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=False, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

### End of Evening Process